## mbti-random-forest

### 1 Import packages and load preprocessed dataframe

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import pickle as pk
import time
from pprint import pprint
from tqdm import tqdm
from joblib import Parallel, delayed
from utilities import clean_posts, postVectorizer
from RF import RandomForest, cross_validation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from xgboost import XGBClassifier,plot_importance

method_dict = {
    'RF': 'RandomForest',
    'SVM': 'SVM',
    'XGB': 'XGBoost',
    'DL': 'DeepLearning'
}

type_dict = {
    0: ['I', 'N', 'T', 'J'],
    1: ['E', 'S', 'F', 'P']
}

with open('pickles/type_explanation.pk', 'rb') as pkl:
    type_explanation = pk.load(pkl)
    
type_keys = list(type_explanation.keys())

In [2]:
with open('pickles/tfidf_df.pk', 'rb') as pkl:
    tfidf_df = pk.load(pkl)

with open('pickles/df.pk', 'rb') as pkl:
    df = pk.load(pkl)

with open('pickles/clf_ie.pk', 'rb') as pkl:
    clf_ie = pk.load(pkl)

with open('pickles/clf_ns.pk', 'rb') as pkl:
    clf_ns = pk.load(pkl)

with open('pickles/clf_tf.pk', 'rb') as pkl:
    clf_tf = pk.load(pkl)

with open('pickles/clf_jp.pk', 'rb') as pkl:
    clf_jp = pk.load(pkl)

### 2 Training and Benchmarking

In [3]:
# train_by_type
# @params:
#     _type: (string), type to be classified; types:[IE, NS, TF, JP]
#    method: (string), type of the classifier; methods:['RF', 'SVM', 'XGB']
# benchmark: (boolean), whether benchmark on dataset; benchmark:[True, False]
##

def train_by_type(_type, method='RF', benchmark=False):
    print(">>> Training Type {}".format(_type))
    y = df[_type].values.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(tfidf_df.values, y, test_size=0.2, train_size=0.8,
                                                        random_state=None, shuffle=True, stratify=None)
    
    print("# @Training START #")
    if method == 'RF':
        clf = RandomForest(n_estimators=100, verbose=0, min_leaf_size=5)
        clf.fit(np.concatenate((X_train, y_train), axis=1))
    
    elif method == 'SVM':
        clf = SVC(gamma='auto', probability=True)
        clf.fit(X_train, y_train)
    
    elif method == 'XGB':
        clf = XGBClassifier()
        clf.fit(X_train, y_train, 
                early_stopping_rounds=10,
                eval_metric="logloss",
                eval_set=[(X_test, y_test)],
                verbose=False)
    else:
        raise ValueError("Invalid Method.")
        
    print("# @Training END #\n")
    
    time.sleep(0.5)
    
    if benchmark:
        print("# @Scoring START # --- {:s}".format(method_dict[method]))
        time.sleep(0.5)
        print("Type: {}: {} : {} = {} : {}".format(_type, _type[0], _type[1], sum(y)/len(y),1-sum(y)/len(y)))
        time.sleep(0.5)
        pred_train = [clf.predict(i.reshape(1,-1)) for i in tqdm(X_train)]
        print("Accuracy on training set - %s" % _type, accuracy_score(y_train, pred_train))
        print("F1 Score on training set - %s" % _type, \
              (f1_score(y_train, pred_train) if len(set(pred_train))>1 else 0.0))
        time.sleep(0.5)
        pred_test = [clf.predict(i.reshape(1,-1)) for i in tqdm(X_test)]
        print("Accuracy on testing set - %s" % _type, accuracy_score(y_test, pred_test))
        print("F1 Score on testing set - %s" % _type, \
              (f1_score(y_test, pred_test) if len(set(pred_test))>1 else 0.0))
        print("# @Scoring END #\n")
    
    return clf

In [4]:
# Separately benchmarking
clf_ie = train_by_type('IE', 'RF', benchmark=True)
clf_ns = train_by_type('NS', 'RF', benchmark=True)
clf_tf = train_by_type('TF', 'RF', benchmark=True)
clf_jp = train_by_type('JP', 'RF', benchmark=True)

>>> Training Type IE
# @Training START #
# @Training END #

# @Scoring START # --- RandomForest
Type: IE: I : E = [0.23043228] : [0.76956772]


100%|██████████| 6940/6940 [00:34<00:00, 200.69it/s]


Accuracy on training set - IE 0.9487031700288184
F1 Score on training set - IE 0.8764746703678001


100%|██████████| 1735/1735 [00:08<00:00, 209.52it/s]


Accuracy on testing set - IE 0.7809798270893372
F1 Score on testing set - IE 0.0
# @Scoring END #

>>> Training Type NS
# @Training START #
# @Training END #

# @Scoring START # --- RandomForest
Type: NS: N : S = [0.13798271] : [0.86201729]


...

Accuracy on testing set - TF 0.7475504322766571
F1 Score on testing set - TF 0.7684989429175476
# @Scoring END #

>>> Training Type JP
# @Training START #
# @Training END #

# @Scoring START # --- RandomForest
Type: JP: J : P = [0.60414986] : [0.39585014]


100%|██████████| 6940/6940 [00:32<00:00, 210.60it/s]


Accuracy on training set - JP 0.9995677233429395
F1 Score on training set - JP 0.9996401583303347


100%|██████████| 1735/1735 [00:08<00:00, 212.17it/s]

Accuracy on testing set - JP 0.6426512968299711
F1 Score on testing set - JP 0.7591297591297591
# @Scoring END #



In [5]:
# Store clfs
with open('pickles/clf_ie.pk', 'wb') as pkl:
    pk.dump(clf_ie, pkl)

with open('pickles/clf_ns.pk', 'wb') as pkl:
    pk.dump(clf_ns, pkl)

with open('pickles/clf_tf.pk', 'wb') as pkl:
    pk.dump(clf_tf, pkl)

with open('pickles/clf_jp.pk', 'wb') as pkl:
    pk.dump(clf_jp, pkl)

In [6]:
# predict_full_type
# @params:
#      text: (string), text(post) to predict.
#     _type: (string or None), True type; types:[INTJ~ESFP],
#                              if None: predict a non-recorde sample.
#    strict: (boolean), if True:
#                           return a tuple of index (predicted, true).
#                       if False:
#                           return a match rate determined by each subtype;
#                           e.g.: INTJ - INTP : 75% matched.
#                       benchmark:[True, False]
##
def predict_full_type(text, _type=None, strict=True):
    text = postVectorizer(clean_posts(text))
    IE = clf_ie.predict(text)
    NS = clf_ns.predict(text)
    TF = clf_tf.predict(text)
    JP = clf_jp.predict(text)
    match_rate = 0
    pred_type = type_dict[IE][0] + type_dict[NS][1] + type_dict[TF][2] + type_dict[JP][3]

    if _type is None:
        return pred_type
    
    if strict:    
        return type_keys.index(pred_type), type_keys.index(_type)
    else:
        for i in range(4):
            if _type[i] == pred_type[i]:
                match_rate += 25
        print("Predicted Type: {}  |  True Type: {}  |  [{}{}%]Matched".format(
            pred_type, _type, "" if match_rate==100 else " ", match_rate))
        return match_rate

In [7]:
# Overall benchmarking strictly

test_size = df.shape[0]

preds = Parallel(n_jobs=4, prefer="threads")\
(delayed(predict_full_type)(df.sep_posts[i], df.type[i], strict=True) for i in tqdm(range(test_size)))

print("Overall Accuracy: {}".format(accuracy_score([i[1] for i in preds], [i[0] for i in preds])))
print("Overall F1 Score: {}".format(f1_score([i[1] for i in preds], [i[0] for i in preds], average='macro')))

100%|██████████| 8675/8675 [10:20<00:00, 13.98it/s]


Overall Accuracy: 0.7068587896253602
Overall F1 Score: 0.42291591900581316


In [8]:
# Overall benchmarking non-strictly
test_size = df.shape[0]
acc = Parallel(n_jobs=4, prefer="threads")\
(delayed(predict_full_type)(df.sep_posts[i], df.type[i], strict=False) for i in tqdm(range(test_size)))

print("Overall Accuracy: {}".format(np.sum(acc) / (100*test_size)))

  0%|          | 12/8675 [00:00<06:17, 22.92it/s]

Predicted Type: INTP  |  True Type: INTP  |  [100%]Matched
Predicted Type: INFJ  |  True Type: INFJ  |  [100%]Matched
Predicted Type: INTJ  |  True Type: INTJ  |  [100%]Matched
Predicted Type: ENTP  |  True Type: ENTP  |  [100%]Matched
Predicted Type: INTJ  |  True Type: INTJ  |  [100%]Matched
Predicted Type: INFJ  |  True Type: INFJ  |  [100%]Matched


  0%|          | 15/8675 [00:00<06:40, 21.61it/s]

Predicted Type: ENTJ  |  True Type: ENTJ  |  [100%]Matched
Predicted Type: INTJ  |  True Type: INTJ  |  [100%]Matched
Predicted Type: INTP  |  True Type: INTP  |  [100%]Matched


  0%|          | 19/8675 [00:00<08:33, 16.87it/s]

...

100%|█████████▉| 8670/8675 [10:28<00:00, 12.86it/s]

Predicted Type: INTP  |  True Type: ENTP  |  [ 75%]Matched
Predicted Type: INTP  |  True Type: INTJ  |  [ 75%]Matched
Predicted Type: INTP  |  True Type: INTP  |  [100%]Matched
Predicted Type: INTP  |  True Type: INTP  |  [100%]Matched


100%|██████████| 8675/8675 [10:28<00:00, 13.80it/s]

Predicted Type: ENTP  |  True Type: ENTP  |  [100%]Matched
Predicted Type: ENTP  |  True Type: ENTP  |  [100%]Matched
Predicted Type: INTJ  |  True Type: INTJ  |  [100%]Matched


Predicted Type: INTJ  |  True Type: INFJ  |  [ 75%]MatchedPredicted Type: INTJ  |  True Type: INTJ  |  [100%]Matched

Predicted Type: INFP  |  True Type: ENFP  |  [ 75%]Matched
Predicted Type: INFP  |  True Type: ISFP  |  [ 75%]Matched
Predicted Type: INFP  |  True Type: INFP  |  [100%]Matched
Predicted Type: INTP  |  True Type: INTP  |  [100%]Matched
Predicted Type: INFP  |  True Type: INFP  |  [100%]Matched
Overall Accuracy: 0.917463976945245


### 3 Further study on Kaggle ForumMessage

In [9]:
k_data = pd.read_csv('../dataset/ForumMessages.csv').Message.dropna().values
k_texts = []
for i in k_data:
    if len(i) > 1000:
        k_texts.append(i)

In [10]:
def k_clean(texts):
    texts = [re.sub(r'<code>.*</code>', " ", s) for s in texts]
    texts = [re.sub(r'<[/]?[a-z]+>', "", s) for s in texts]
    return texts

k_texts = k_clean(k_texts)

In [12]:
res = predict_full_type(k_texts[5], _type=None)
pprint(k_texts[1])
print(res)
pprint(type_explanation[res])

('Hi All,\r\n'
 'I have a question about cross-validation. I am fitting a glm R model to my '
 'training dataset and it gives me an AUC ~ 0.98. Then I do a 5-fold '
 'cross-validation. Each of my cross validations comes out with a AUC ~0.98 '
 'but then when I submit my model, it has\r\n'
 ' a AUC~0.6 on leaderboard. What am I doing wrong? If I am overfitting so '
 "badly on training set, I don't understand why doesn't cross validation show "
 'that? Here is my code for cross-validation.\r\n'
 '\r\n'
 'data is a dataframe\r\n'
 'k is number of folds.\r\n'
 '----------\r\n'
 'kfold.glm&lt;-function(data,k)\r\n'
 '{\r\n'
 'n&lt;-as.integer(nrow(data)/k)\r\n'
 'err.vect&lt;-rep(NA,k)\r\n'
 'for (i in 1:k)\r\n'
 '{\r\n'
 's1&lt;-((i-1)*n&#43;1)\r\n'
 's2&lt;-(i*n)\r\n'
 'subset&lt;-s1:s2\r\n'
 'train&lt;-data[-subset,]\r\n'
 'test&lt;-data[subset,]\r\n'
 'fit &lt;- glm(action ~.,data=train,family=&quot;binomial&quot;)\r\n'
 'prediction &lt;- predict(fit,newdata=test,type=&quot;response&quo

## END
@ Karl